In [62]:
import pandas as pd
from riotwatcher import RiotWatcher

In [20]:
rw = RiotWatcher("API_KEY")

In [35]:
mh = rw.match.matchlist_by_account(end_index=20, region='EUW1', account_id=acc_id)

In [118]:
game = rw.match.by_id(match_id=3511957748, region="EUW1")

In [115]:
def game_to_dataframe(game, custom_names=None, team_names=None, custom_positions=None):
    ps_ids_df = game_participant_ids_to_dataframe(game['participantIdentities'])
    ps_df = game_participants_to_dataframe(game['participants'])
    df_result = pd.concat([ps_ids_df, ps_df], axis=1)
    
    if custom_names:
        df_result['player_name'] = custom_names
        
    if team_names:
        df_result.loc[:5, 'team_name'] = team_names[0]
        df_result.loc[5:10, 'team_name'] = team_names[1]
        
    if custom_positions:
        df_result['position'] = c_positions
        
    return df_result

In [90]:
def game_participants_to_dataframe(participants):
    stats = [p['stats'] for p in participants]
    timeline = [p['timeline'] for p in participants]
    [p.pop('stats') for p in participants] # remove stats
    [p.pop('timeline') for p in participants] # remove timeline
    df1 = pd.concat([pd.DataFrame(p, index=(i,)) for i, p in enumerate(participants)])
    df2 = pd.concat([pd.DataFrame(s, index=(i,)) for i, s in enumerate(stats)])
    df3 = pd.concat([match_timeline_to_dataframe(t) for i, t in enumerate(timeline)])
    return pd.concat([df1, df2, df3], axis=1)

In [100]:
def game_participant_ids_to_dataframe(participant_ids):
    return pd.concat([pd.DataFrame(p_id, index=(i,)) for i, p_id in enumerate(pidtts)])

In [113]:
c_names = ["Moryo", "RafaL0L", "Ripi", "Monk", "Tasteless", "Kektz", "Lamabear", "Pretty", "Sedrion", "Anthrax"]
t_names = ["EMZ", "ASUS"]
c_positions = ['TOP', 'JUNG', 'MID', 'ADC', 'SUPP', 'TOP', 'JUNG', 'MID', 'ADC', 'SUPP']

In [119]:
game_to_dataframe(game, custom_names=c_names, team_names=t_names, custom_positions=c_positions).T.drop_duplicates().T

,participantId,championId,highestAchievedSeasonTier,spell1Id,spell2Id,teamId,assists,champLevel,combatPlayerScore,damageDealtToObjectives,...,xpdiffpm@20,xpdiffpm@30,xpdiffpm@end,gpm@10,gpm@20,gpm@30,gpm@end,player_name,team_name,position
0,1,8,MASTER,4,7,100,3,18,0,6355,...,57.8,-116.3,-72,267.1,412,356.9,409.2,Moryo,EMZ,TOP
1,2,154,MASTER,4,11,100,5,15,0,16973,...,-106.5,-15.9,192,244.1,328.7,248.7,346.6,RafaL0L,EMZ,JUNG
2,3,142,CHALLENGER,7,4,100,7,18,0,14636,...,76.5,-241,267.6,311.9,405.3,369.1,628.2,Ripi,EMZ,MID
3,4,51,UNRANKED,7,4,100,6,17,0,33511,...,24.2,27.35,36.6,290.8,455.1,465.9,599.4,Monk,EMZ,ADC
4,5,201,MASTER,4,14,100,10,15,0,7143,...,24.2,27.35,36.6,198.3,262.9,227.3,319.6,Tasteless,EMZ,SUPP
5,6,126,CHALLENGER,4,12,200,1,18,0,15196,...,-57.8,116.3,72,299.1,377,521.3,436.4,Kektz,ASUS,TOP
6,7,113,UNRANKED,11,4,200,6,15,0,4808,...,106.5,15.9,-192,230.3,307.8,336.1,207,Lamabear,ASUS,JUNG
7,8,42,CHALLENGER,4,12,200,5,18,0,7352,...,-76.5,241,-267.6,275.6,370.3,543.1,269.2,Pretty,ASUS,MID
8,9,96,DIAMOND,4,7,200,2,16,0,9870,...,-24.2,-27.35,-36.6,267.3,344.5,588.2,398,Sedrion,ASUS,ADC
9,10,432,MASTER,4,14,200,7,14,0,2266,...,-24.2,-27.35,-36.6,151.3,208.1,318.4,235.4,Anthrax,ASUS,SUPP


In [73]:
def match_timeline_to_dataframe(timeline):
    columns = ["lane", "role", "participantId", "cspm@10", "cspm@20", "cspm@30", "cspm@end",
               "csdiffpm@10", "csdiffpm@20", "csdiffpm@30", "csdiffpm@end", "dmgtpm@10",
               "dmgtpm@20", "dmgtpm@30", "dmgtpm@end", "dmgtdiffpm@10", "dmgtdiffpm@20",
               "dmgtdiffpm@30", "dmgtdiffpm@end", "xppm@10", "xppm@20", "xppm@30",
               "xppm@end", "xpdiffpm@10", "xpdiffpm@20", "xpdiffpm@30", "xpdiffpm@end"]
    participant_id = timeline["participantId"]
    tl_df = pd.DataFrame(index=(participant_id - 1,), columns=columns)
    try:
        tl_df["cspm@10"] = timeline["creepsPerMinDeltas"]["0-10"]
    except:
        tl_df["cspm@10"] = None
    try:
        tl_df["cspm@20"] = timeline["creepsPerMinDeltas"]["10-20"]
    except:
        tl_df["cspm@20"] = None
    try:
        tl_df["cspm@30"] = timeline["creepsPerMinDeltas"]["20-30"]
    except:
        tl_df["cspm@30"] = None
    try:
        tl_df["cspm@end"] = timeline["creepsPerMinDeltas"]["30-end"]
    except:
        tl_df["cspm@end"] = None
    try:
        tl_df["csdiffpm@10"] = timeline["csDiffPerMinDeltas"]["0-10"]
    except:
        tl_df["csdiffpm@10"] = None
    try:
        tl_df["csdiffpm@20"] = timeline["csDiffPerMinDeltas"]["10-20"]
    except:
        tl_df["csdiffpm@20"] = None
    try:
        tl_df["csdiffpm@30"] = timeline["csDiffPerMinDeltas"]["20-30"]
    except:
        tl_df["csdiffpm@30"] = None
    try:
        tl_df["csdiffpm@end"] = timeline["csDiffPerMinDeltas"]["30-end"]
    except:
        tl_df["csdiffpm@end"] = None
    try:
        tl_df["dmgtpm@10"] = timeline["damageTakenPerMinDeltas"]["0-10"]
    except:
        tl_df["dmgtpm@10"] = None
    try:
        tl_df["dmgtpm@20"] = timeline["damageTakenPerMinDeltas"]["10-20"]
    except:
        tl_df["dmgtpm@20"] = None
    try:
        tl_df["dmgtpm@30"] = timeline["damageTakenPerMinDeltas"]["20-30"]
    except:
        tl_df["dmgtpm@30"] = None
    try:
        tl_df["dmgtpm@end"] = timeline["damageTakenPerMinDeltas"]["30-end"]
    except:
        tl_df["dmgtpm@end"] = None
    try:
        tl_df["dmgtdiffpm@10"] = timeline["damageTakenDiffPerMinDeltas"]["0-10"]
    except:
        tl_df["dmgtdiffpm@10"] = None
    try:
        tl_df["dmgtdiffpm@20"] = timeline["damageTakenDiffPerMinDeltas"]["10-20"]
    except:
        tl_df["dmgtdiffpm@20"] = None
    try:
        tl_df["dmgtdiffpm@30"] = timeline["damageTakenDiffPerMinDeltas"]["20-30"]
    except:
        tl_df["dmgtdiffpm@30"] = None
    try:
        tl_df["dmgtdiffpm@end"] = timeline["damageTakenDiffPerMinDeltas"]["30-end"]
    except:
        tl_df["dmgtdiffpm@end"] = None
    try:
        tl_df["gpm@10"] = timeline["goldPerMinDeltas"]["0-10"]
    except:
        tl_df["gpm@10"] = None
    try:
        tl_df["gpm@20"] = timeline["goldPerMinDeltas"]["10-20"]
    except:
        tl_df["gpm@20"] = None
    try:
        tl_df["gpm@30"] = timeline["goldPerMinDeltas"]["20-30"]
    except:
        tl_df["gpm@30"] = None
    try:
        tl_df["gpm@end"] = timeline["goldPerMinDeltas"]["30-end"]
    except:
        tl_df["gpm@end"] = None
    try:
        tl_df["xppm@10"] = timeline["xpPerMinDeltas"]["0-10"]
    except:
        tl_df["xppm@10"] = None
    try:
        tl_df["xppm@20"] = timeline["xpPerMinDeltas"]["10-20"]
    except:
        tl_df["xppm@20"] = None
    try:
        tl_df["xppm@30"] = timeline["xpPerMinDeltas"]["20-30"]
    except:
        tl_df["xppm@30"] = None
    try:
        tl_df["xppm@end"] = timeline["xpPerMinDeltas"]["30-end"]
    except:
        tl_df["xppm@end"] = None
    try:
        tl_df["xpdiffpm@10"] = timeline["xpDiffPerMinDeltas"]["0-10"]
    except:
        tl_df["xpdiffpm@10"] = None
    try:
        tl_df["xpdiffpm@20"] = timeline["xpDiffPerMinDeltas"]["10-20"]
    except:
        tl_df["xpdiffpm@20"] = None
    try:
        tl_df["xpdiffpm@30"] = timeline["xpDiffPerMinDeltas"]["20-30"]
    except:
        tl_df["xpdiffpm@30"] = None
    try:
        tl_df["xpdiffpm@end"] = timeline["xpDiffPerMinDeltas"]["30-end"]
    except:
        tl_df["xpdiffpm@end"] = None
    tl_df["lane"] = timeline["lane"]
    tl_df["role"] = timeline["role"]
    tl_df["participantId"] = participant_id

    return tl_df